# **Exploration des données**

In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = "../data/online_retail_II.xlsx"

df = pd.read_excel(file_path, sheet_name="Year 2009-2010")

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [ ]:
#Combien de lignes et de colonnes
print(f"Format du dataset: {df.shape}")

Format du dataset: (525461, 8)


In [4]:
#Y a-t-il des données manquantes ?
print("\nDonnées manquantes par colonne:")
print(df.isnull().sum())


Données manquantes par colonne:
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [6]:
107927/525461

0.20539488182757618

In [ ]:
#On remarque que nos valeurs minimales dans Quantity et Price sont négatives. Ca peut biaiser nos calculs. 
stats = df[['Quantity', 'Price']].describe()
print(stats)

            Quantity          Price
count  525461.000000  525461.000000
mean       10.337667       4.688834
std       107.424110     146.126914
min     -9600.000000  -53594.360000
25%         1.000000       1.250000
50%         3.000000       2.100000
75%        10.000000       4.210000
max     19152.000000   25111.090000


In [12]:
# Étape A : On ne garde que les ventes réelles (Quantité strictement positive)
# Pourquoi ? On veut prévoir la demande, pas gérer les retours pour l'instant.
df_clean = df[df['Quantity']>0]

# Étape B : On ne garde que les produits avec un prix positif
# Pourquoi ? Les prix à 0 ou négatifs sont des erreurs ou des cadeaux, pas des ventes répétables.
df_clean = df_clean[df['Price']>0]

# Étape C : On vérifie le résultat
print(f'Ancien nombre de ligne : {len(df)}')
print(f'Nouveau nombre de lignes : {len(df_clean)}')

Ancien nombre de ligne : 525461
Nouveau nombre de lignes : 511566


C:\Users\Alexis\AppData\Local\Temp\ipykernel_18076\2991879355.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_clean = df_clean[df['Price']>0]


In [15]:
# Objectif : Calculer la valeur financière de chaque transaction
# Pourquoi ? On ne gère pas de la même manière une vis à 0.01€ et un moteur à 5000€
df_clean['Total_Line'] = df_clean['Quantity']*df_clean['Price']

df_clean.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Total_Line
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [21]:
# Objectif : Sommer les ventes par produit (StockCode)
# Pourquoi ? Pour identifier l'importance relative de chaque référence

product_analysis = df_clean.groupby('StockCode').agg({
    'Quantity': 'sum',
    'Total_Line': 'sum',
    'Description': 'first'
}).reset_index()

# On trie par valeur décroissante pour voir les "Top" produits
product_analysis = product_analysis.sort_values(by='Total_Line', ascending=False)

# On affiche le Top 10
product_analysis.head(10)

,StockCode,Quantity,Total_Line,Description
4236,M,2827,262979.58,Manual
1511,22423,13698,170078.51,REGENCY CAKESTAND 3 TIER
3796,85123A,58487,158590.87,WHITE HANGING HEART T-LIGHT HOLDER
4235,DOT,730,116408.71,DOTCOM POSTAGE
3781,85099B,49875,89114.78,JUMBO BAG RED WHITE SPOTTY
2501,84879,45348,73092.99,ASSORTED COLOUR BIRD ORNAMENT
1199,22086,17226,58189.25,PAPER CHAIN KIT 50'S CHRISTMAS
2186,47566,10083,49682.72,PARTY BUNTING
4238,POST,2310,49477.54,POSTAGE
2348,84347,23049,47985.09,ROTATING SILVER ANGELS T-LIGHT HLDR


In [22]:
non_stock_code = ['POST', 'D', 'M', 'BANK CHARGES', 'DOT', 'CRUK']

product_analysis = product_analysis[~product_analysis['StockCode'].isin(non_stock_code)]

product_analysis.head(10)

,StockCode,Quantity,Total_Line,Description
1511,22423,13698,170078.51,REGENCY CAKESTAND 3 TIER
3796,85123A,58487,158590.87,WHITE HANGING HEART T-LIGHT HOLDER
3781,85099B,49875,89114.78,JUMBO BAG RED WHITE SPOTTY
2501,84879,45348,73092.99,ASSORTED COLOUR BIRD ORNAMENT
1199,22086,17226,58189.25,PAPER CHAIN KIT 50'S CHRISTMAS
2186,47566,10083,49682.72,PARTY BUNTING
2348,84347,23049,47985.09,ROTATING SILVER ANGELS T-LIGHT HLDR
1022,21843,4277,45254.95,RETRO SPOT CAKE STAND
2199,48138,6413,42148.03,DOOR MAT UNION FLAG
101,20685,5882,39777.02,RED SPOTTY COIR DOORMAT


In [23]:
#Calcul de la part de chaque produit dans le CA Total
total_revenue = product_analysis['Total_Line'].sum()
product_analysis['Revenue_Share'] = product_analysis['Total_Line'] / total_revenue

#Calcul du cumulé (Somme progressive)
#L'objectif est de voir à partir de quelle ligne on atteint 80% du CA total

product_analysis['Cumulative_Share'] = product_analysis['Revenue_Share'].cumsum()

#Attribution des classes ABC
# Classe A : Les produits faisant les premiers 80% du CA
# Classe B : Les 15% suivants (jusqu'à 95%)
# Classe C : Le reste (le "fond de catalogue")

def assign_abc(cum_share):
    if cum_share <= 0.80:
        return 'A'
    elif cum_share <= 0.95:
        return 'B'
    else: 
        return 'C'
    
product_analysis['ABC_Class'] = product_analysis['Cumulative_Share'].apply(assign_abc)

# On regarde combien on a de produits dans chaque classe
print(product_analysis['ABC_Class'].value_counts())

ABC_Class
C    2230
B    1092
A     924
Name: count, dtype: int64
